In [ ]:

# Load the dataset
with open("Sequential_Dataset.txt", "r") as file:
    dataset = file.read()

# Total number of characters
total_characters = len(dataset)

# Unique characters and their counts
unique_characters = set(dataset)
char_counts = {char: dataset.count(char) for char in unique_characters}

# Most frequent characters (Top 10)
sorted_char_counts = sorted(char_counts.items(), key=lambda x: x[1], reverse=True)[:10]

# Define sequence length
sequence_length = 20

# Create input-output pairs
inputs = []
outputs = []

for i in range(0, len(dataset) - sequence_length):
    inputs.append(dataset[i:i+sequence_length])
    outputs.append(dataset[i+sequence_length])

# Create character to integer mapping
char_to_int = {char: i for i, char in enumerate(sorted(unique_characters))}
int_to_char = {i: char for char, i in char_to_int.items()}

# One-hot encode the input sequences and outputs
import numpy as np

encoded_inputs = []
for sequence in inputs:
    encoded_inputs.append([char_to_int[char] for char in sequence])
    
encoded_outputs = [char_to_int[char] for char in outputs]

X = np.zeros((len(encoded_inputs), sequence_length, len(unique_characters)), dtype=np.bool)
y = np.zeros((len(encoded_outputs), len(unique_characters)), dtype=np.bool)

for i, sequence in enumerate(encoded_inputs):
    for t, char_int in enumerate(sequence):
        X[i, t, char_int] = 1
    y[i, encoded_outputs[i]] = 1


In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the model
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, len(unique_characters))))
model.add(Dense(len(unique_characters), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
model.summary()


In [ ]:

# Train the model
model.fit(X, y, epochs=15, batch_size=150)


In [ ]:

# Choose 5th training record for prediction
input_sequence = X[4]
predicted = model.predict(input_sequence.reshape(1, sequence_length, len(unique_characters)))
predicted_char = int_to_char[np.argmax(predicted)]

# Print whole sequence including predicted characters
sequence_with_prediction = inputs[4] + predicted_char
sequence_with_prediction
